In [1]:
from scribe_classifier.data.canada import AllCodes, TitleSet, SimpleModel, CombinedModels
from sklearn import metrics

In [2]:
all_codes = AllCodes.load_from_pickle("./source_data/pickles/canada/tidy_sets/all_codes.P", is_path=True)

target_level = 3
emptyset_label = "NA"

mdl_sgd = SimpleModel.load_from_pickle(
    "./source_data/pickles/canada/trained_models/simple.lvl%d.P" % target_level,
    is_path=True
)
mdl_nb = SimpleModel.load_from_pickle(
    "./source_data/pickles/canada/trained_models/simple.lvl%d.bayes.P" % target_level,
    is_path=True
)

In [3]:
test = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/test.set.lvl%d.P' % target_level, is_path=True)
valid = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/valid.set.lvl%d.P' % target_level, is_path=True)
train = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/train.set.lvl%d.P' % target_level, is_path=True)

valid = valid.copy_and_append_empty_string_class(label=emptyset_label, prop_records=0.25)
test = test.copy_and_append_empty_string_class(label=emptyset_label, prop_records=0.25)

In [ ]:
cmb_mdl = CombinedModels(
    target_level=target_level, 
    emptyset_label="NA",
    trained_simple_sgd=mdl_sgd, 
    trained_simple_multinom_nb=mdl_nb, 
    all_codes=all_codes)
cmb_mdl.fit_titleset(train)

In [ ]:
valid_pred = cmb_mdl.predict_titleset(valid)
test_pred = cmb_mdl.predict_titleset(test)
print(valid_pred)
# print(valid.get_code_vec(target_level=target_level))
print(test_pred)
# print(test.get_code_vec(target_level=target_level))

In [ ]:
print("Validation Set:")
print(metrics.classification_report(valid.get_code_vec(target_level=target_level), valid_pred))
# print(metrics.confusion_matrix(valid.Y, valid_pred))

In [ ]:
print("Test Set:")
print(metrics.classification_report(test.get_code_vec(target_level=target_level), test_pred))